<a href="https://githubtocolab.com/Eunseob/purdue_me597/blob/main/lab/lab4/L4_Colab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 4.1 Middleware 1 - MTConnect Agent

## Learning Goals

Students will be able to:


1. Execute basic commands in using MTConnect architecture
2. Implement connections to datastream from MTConnect adapter to MTConnect agent

## 1.1 Introduction

MTConnect defines the communications and protocols, and it provides the vocabulary and semantics of the information models. MTConnect agent is a form of middleware that provides a service to client applications that require data from specified manufacturing equipment or sensors. The agent needs to receive a stream of data directly from a compliant device or from a software adapter. The data stream is received by the agent and buffered in a circular buffer. Upon request (HTTP) from a client, the agent binds the buffered data to the device data model (device.xml) and provides the appropriate response XML (Extensible Markup Language) document (*.xml). 

Typical MTConnect architecture is shown in Figure 1. There are 3 different cases of MTConnect architecture. 
Of course, mixed configurations from each case are applicable. An agent can support multiple machines. Adapters can be provided by the manufacturer of the machine and sensor or developed. In addition, adapter can reside 1) on the machine’s controller, 2) connected to the PLC wires and controller, and 3) remotely if the machine communicates using a network protocol. Applications only communicate with MTConnect agent. The data streamflow of MTConnect is unidirectional from the device to the application. It means that a client application is only accessible to read-only data from MTConnect agent. MTConnect does not allow to write data to MTConnect agent by a client. We may think a lot of applications such as real-time web-based monitoring, creating digital twins, data analytics, and so on. Therefore, the applications are differed by demands. 

The purpose of Lab4 is to create a basic MTConnect agent stream in various network configurations and to request MTconnect agent data as a client application. In addition, by simulating MTConnect adapter, we will make data stream for MTConnect agent to understand and practice it. 

<img src="https://github.com/Eunseob/purdue_me597/blob/main/lab/img/lab4_fig1.png?raw=true" width="70%">

*Figure 1 MTConnect architecture*


## 1.2 MTConnect Agent Overview


Compiled MTConnect agent execution files upon the operating system and computer architecture and required files are prepared on [Github repo](https://github.com/Eunseob/purdue_me597/tree/main/lab/lab4/MTConnect_agent). Below is the download link for the MTConnect agent for this lab depending on OS.

  * [agent_Windows](https://github.com/Eunseob/purdue_me597/blob/main/lab/lab4/MTConnect_agent/agent_Windows.zip?raw=true)

  * [agent_Raspberry Pi](https://github.com/Eunseob/purdue_me597/blob/main/lab/lab4/MTConnect_agent/agent_RaspberryPi.zip?raw=true)

  * [agent_Ubuntu](https://github.com/Eunseob/purdue_me597/blob/main/lab/lab4/MTConnect_agent/agent_Ubuntu.zip?raw=true)

  * [agent_macOS](https://github.com/Eunseob/purdue_me597/blob/main/lab/lab4/MTConnect_agent/agent_macOS.zip?raw=true)

If you’d like to compile MTConnect agent execution file programmed by CMake yourself, please visit https://github.com/mtconnect/cppagent and follow the directions.  
Please download ‘agent_Windows.zip’ and ‘agent_RaspberryPi.zip’ from Brightspace on laptop and 
Raspberry Pi, respectively. And then, extract (unzip) each compressed ‘zip’ file on each computer. Note that if your laptop is different OS, download appropriate MTConnect agent from the link above. You will see the files as Figure 2. The explanations about each sub-directory and file are as follows: 

<img src="https://github.com/Eunseob/purdue_me597/blob/main/lab/img/lab4_fig2.png?raw=true" width="100%">


*Figure 2 Contents in agent directory: Windows (left) and Raspberry Pi (right)*

1.	**‘schemas’** directory 

  a.	It contains MTConnect XML semantics and schema according to the standards. 

  b.	You do not need to touch or change this directory. 
2.	**‘styles’** directory 

  a.	It contains the files, such as icon image, CSS, XSL, required for web-browser visualization when you request data from MTConnect agent. 

  b.	You do not need to touch or change this directory. 
3. **‘agent.cfg’**

  a.	It is the agent configuration file that provides service configuration information. 

  b.	Agent execution file (‘agent.exe’ or ‘agent’ (without extension in case of Raspberry Pi)) uses onstart to discover the appropriate pieces of equipment or device it is to connect to (adapters). 

  c.	It also determines how to log data, the location of the device data model, and the port of adapter it will host the web services on. 

  d.	IMPORTANT: You **MUST** modify this file upon the network, adapter device, agent configurations, and so on before you start MTConnect service.  
4. **‘agent.exe’** (Windows) or **agent’** (Raspberry Pi, without extension) 

  a.	This is the compiled agent execution file. 

  b.	IMPORTANT: You can start MTConnect agent service by running this file. 
5. **‘agent.log’**

  a.	This is log file of the agent and automatically generated by the agent service. 

  b.	If you have any issues when running MTConnect agent, you need to take a look at the log files. 
6. **‘Device.xml’**

  a.	It provides an XML schema and device data model for a particular piece of manufacturing equipment or device. 

  b.	The device data model defines the complete composition of a piece of equipment based on its components and the dataitems’ generated by each component. 

  c.	IMPORTANT: You **MUST** modify this file upon data configurations and the MTConnect standards before you start MTConnect service. 

Therefore, when you start MTConnect agent service, you **MUST** modify *‘agent.cfg’* and *‘Device.xml’*. You can modify these files using any text editors, Notepad, Notepad++, WordPad and so on. Basic and required descriptions about *‘agent.cfg’* (Figure 3) file are as follows. Please refer to MTConnect standard document and https://github.com/mtconnect/cppagent for other parameters.


1.	(Line 1) Devices = 

  a.	This points out the XML document for equipment configurations and data schema. 

  b.	In this example, the device XML document filename is ‘Device.xml’ in the same directory. 
2.	(Line 2) Port = 

  a.	This means port number of MTConnect agent service. 

  b.	Default port number of MTConnect agent service is 5000. 
3.	(Line 5) BufferSize = 

  a.	This defines the total length of the sequence (buffer) of the agent as the power of 2 of the number. 

  b.	Default is 17 that means the maximum sequence which agent can contain is 2<sup>17</sup> (131,072). 

  c.	Maximum buffer size depends on the memory size of the agent computer. 
4.	(Line 10 – 16) Adapters { } 

  a.	This defines MTConnect adapters that are supposed to be connected to MTConnect agent. 

  b.	The agent only receives data streams from adapters indicated here. 
  
  c.	It is also possible to have multiple adapters for an agent. 

  d.	You MUST specify both Host IP address and Port number. 

  e.	Default port number is 7878. 

  f.	When you use multiple adapters, do not use the same port number if the IP address is the same.

<img src="https://github.com/Eunseob/purdue_me597/blob/main/lab/img/lab4_fig3.png?raw=true" width="60%">

*Figure 3 agent.cfg*

Basic descriptions of XML elements of ‘Device.xml’ (Figure 4) are as follows. 

1.	(Line 2) MTConnectDevices 

  a.	It indicates MTConnect information such as schema version, MTConnect version, etc. 

  b.	In most cases, you do not need to modify this element. 
2.	(Line 3) Header 

  a.	It indicates basic information of MTConnect agent by a client such as requested time (creation time), agent computer name (sender), buffer size, MTConnect version, etc. 

  b.	In most cases, you do not need to modify this element. 
3.	(Line 4 – 12) Devices 

  a.	You must modify Devices and its sub-elements. 

  b.	‘Device’ element must have ‘id’ attribute. 

  c.	‘DataItem’ element must have ‘category’, ‘id’, and ‘type’. 

  d.	You can specify other attributes such as ‘name’, ‘units’, ‘subtype’, and so on upon the standard and demands. 

  e.	In this example, Device id is ‘sensor’ and has 2 data items (Acceleration and Temperature). 

  f.	Details of standards and requirements are described in MTConnect standard document. 

<img src="https://github.com/Eunseob/purdue_me597/blob/main/lab/img/lab4_fig4.png?raw=true" width="100%">

*Figure 4 Device.xml*

<br></br>

Please continue to [Lab 4.2 here](https://githubtocolab.com/Eunseob/purdue_me597/blob/main/lab/lab4/L4_Colab2.ipynb).